<a href="https://colab.research.google.com/github/LilyN11/CS136-PSET2/blob/main/experiments/Bar_Plots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys
import json

from pathlib import Path


COLAB_ROOT_PATH = "/content"
IS_COLAB = os.path.exists(COLAB_ROOT_PATH)

if IS_COLAB:
    # Working on Google Colab
    from google.colab import drive

    # Mount Google Drive
    DRIVE_PATH = os.path.join(COLAB_ROOT_PATH, "drive")
    drive.flush_and_unmount()
    drive.mount(DRIVE_PATH)

    # Load config
    CONFIG_PATH = os.path.join(DRIVE_PATH, "MyDrive", "Colab")
    if os.path.exists(os.path.join(CONFIG_PATH, "config.gi.json")):
        with open(os.path.join(CONFIG_PATH, "config.gi.json"), "r") as f:
            config = json.load(f)
    else:
        with open(os.path.join(CONFIG_PATH, "config.json"), "r") as f:
            config = json.load(f)  # fallback to config.json

    # Set up Git credentials
    GIT_USER_NAME = config["GIT_USER_NAME"]
    GIT_TOKEN = config["GIT_TOKEN"]
    GIT_USER_EMAIL = config["GIT_USER_EMAIL"]

    !git config --global user.email {GIT_USER_EMAIL}
    !git config --global user.name {GIT_USER_NAME}

    # Set up project paths
    GIT_OWNER = "tigeryst"
    GIT_REPOSITORY = "comp0258-open-endedness"
    STORAGE_PATH = os.path.join(DRIVE_PATH, "MyDrive", config["DRIVE_PATH"], "Colab")
    ROOT_PATH = os.path.join(COLAB_ROOT_PATH, GIT_REPOSITORY)

    # Clone repo
    GIT_PATH = f"https://{GIT_TOKEN}@github.com/{GIT_OWNER}/{GIT_REPOSITORY}.git"

    if not os.path.exists(ROOT_PATH):
      !git clone "{GIT_PATH}" "{ROOT_PATH}"
    else:
      print(f"Git repo already cloned at {ROOT_PATH}")
      !git -C "{ROOT_PATH}" pull

    # Install dependencies
    %pip install --quiet -r {os.path.join(ROOT_PATH, "colab_requirements.txt")}
    %pip install --quiet -e {ROOT_PATH}
    %pip install --quiet flash-attn --no-build-isolation
else:
    # Working on local machine
    # Get the absolute path of the current file
    current_path = Path().resolve()

    # Traverse upwards to find the directory containing "comp0258"
    ROOT_PATH = None
    for parent in current_path.parents:
        if (
            "comp0258" in parent.name.lower()
        ):  # Match folder name "comp0258" (case-insensitive)
            ROOT_PATH = parent.resolve()
            break

    # If found, print the root path; otherwise, raise an error
    if not ROOT_PATH:
        raise FileNotFoundError("Directory with name 'comp0258...' not found.")

    # Set the storage path to the root path
    STORAGE_PATH = ROOT_PATH
    CONFIG_PATH = ROOT_PATH

# Data and output paths
DATA_PATH = os.path.join(STORAGE_PATH, "data")
OUTPUT_PATH = os.path.join(STORAGE_PATH, "output")
MODEL_PATH = os.path.join(STORAGE_PATH, "models")

if not os.path.exists(DATA_PATH):
    # Create if does not exist
    os.makedirs(DATA_PATH)
    print(f"Created data directory at {DATA_PATH}")

if not os.path.exists(OUTPUT_PATH):
    # Create if does not exist
    os.makedirs(OUTPUT_PATH)
    print(f"Created output directory at {OUTPUT_PATH}")

if not os.path.exists(MODEL_PATH):
    # Create if does not exist
    os.makedirs(MODEL_PATH)
    print(f"Created model directory at {MODEL_PATH}")

# Add root path to sys.path
sys.path.append(ROOT_PATH)

# Load environment variables
from dotenv import load_dotenv

if os.path.exists(os.path.join(CONFIG_PATH, ".env.gi")):
    load_dotenv(os.path.join(CONFIG_PATH, ".env.gi"))
else:
    load_dotenv(os.path.join(CONFIG_PATH, ".env"))  # fallback to .env

print("=" * 50)
print(f"Runtime: {'Google Colab' if IS_COLAB else 'local machine'}")
print(f"CONFIG_PATH: {CONFIG_PATH}")  # where config.gi.json and .env is stored
print(f"ROOT_PATH: {ROOT_PATH}")  # root directory of the project
print(f"STORAGE_PATH: {STORAGE_PATH}")  # where data, output, and models are stored
print(f"DATA_PATH: {DATA_PATH}")
print(f"OUTPUT_PATH: {OUTPUT_PATH}")
print(f"MODEL_PATH: {MODEL_PATH}")
print("=" * 50)

Mounted at /content/drive
Git repo already cloned at /content/comp0258-open-endedness
Already up to date.
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for innovation (pyproject.toml) ... done
Runtime: Google Colab
CONFIG_PATH: /content/drive/MyDrive/Colab
ROOT_PATH: /content/comp0258-open-endedness
STORAGE_PATH: /content/drive/MyDrive/Idea Engine/Colab
DATA_PATH: /content/drive/MyDrive/Idea Engine/Colab/data
OUTPUT_PATH: /content/drive/MyDrive/Idea Engine/Colab/output
MODEL_PATH: /content/drive/MyDrive/Idea Engine/Colab/models


In [12]:
from scipy.cluster.hierarchy import linkage, leaves_list
import neat
import pickle
import plotly.express as px
import toml
import torchvision

from tqdm.notebook import tqdm
tqdm.pandas()

# Custom modules
from innovation.models import (
    cppn_neat as cn
)

import plotly.graph_objects as go
import pandas as pd
import numpy as np

## Configs

In [3]:
# Requires manual config
run_id = "20250329_162400" # run id as current timestamp [yyyyMMdd_HHmmss]

In [4]:
# Automatically determined configs
output_folder = os.path.join(OUTPUT_PATH, run_id)

run_config_path = os.path.join(output_folder, "config.toml")
with open (run_config_path, "r") as f:
    run_config = toml.load(f)

print(f"{run_config=}")

# CPPN-NEAT
neat_config_path = os.path.join(output_folder, "config_neat.ini")

neat_config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    neat_config_path,
)

evo = cn.NeatEvo(neat_config) # initialise evolutionary algorithm

run_config={'run_id': '20250329_162400', 'lvlm_model': 'Qwen/Qwen2.5-VL-7B-Instruct-AWQ', 'n_population': 100, 'n_generation': 2000, 'image_size': 32, 'batch_size': 32, 'max_new_tokens': 50, 'prob_tolerance': 0.2, 'prob_top_n': 50, 'visualize': True, 'visualize_freq': 100, 'confidence_threshold': 0.7, 'n_visualize_sample': 5, 'repo_tag': '1.1.0', 'seed': 42}


## Load Data

In [5]:
imagenet_classes = torchvision.models.AlexNet_Weights.IMAGENET1K_V1.meta["categories"]

with open(os.path.join(output_folder, "classical", "main_df.pkl"), "rb") as f:
    main_df = pickle.load(f)

print(f"{len(imagenet_classes)=}")
print(imagenet_classes[:5])

print(f"{main_df.shape=}")
main_df.sample(5, random_state=run_config["seed"])

len(imagenet_classes)=1000
['tench', 'goldfish', 'great white shark', 'tiger shark', 'hammerhead']
main_df.shape=(2000000, 14)


,elite_id,genome,generation,top_class,top_score,class,score,saved_generation,for_generation,parent_id,parent_genome,parent_generation,parent_top_class,parent_top_score
1828401,67392,Key: 56\nFitness: None\nNodes:\n\t0 DefaultNod...,673,ballpoint,0.370420,fountain pen,0.156515,1828,673.0,44747.0,Key: 56\nFitness: None\nNodes:\n\t0 DefaultNod...,447.0,bannister,0.112638
1200071,119962,Key: 74\nFitness: None\nNodes:\n\t0 DefaultNod...,1199,Bedlington terrier,0.182658,Leonberg,0.010931,1200,1199.0,107344.0,Key: 74\nFitness: None\nNodes:\n\t0 DefaultNod...,1073.0,Mexican hairless,0.115553
194849,8810,Key: 12\nFitness: None\nNodes:\n\t0 DefaultNod...,88,fire screen,0.227457,steel arch bridge,0.092399,194,88.0,8155.0,Key: 12\nFitness: None\nNodes:\n\t0 DefaultNod...,81.0,fire screen,0.171050
1629054,28889,Key: 56\nFitness: None\nNodes:\n\t0 DefaultNod...,288,Granny Smith,0.907194,Granny Smith,0.907194,1629,288.0,14851.0,Key: 56\nFitness: None\nNodes:\n\t0 DefaultNod...,148.0,jellyfish,0.342358
191144,18932,Key: 30\nFitness: None\nNodes:\n\t0 DefaultNod...,189,measuring cup,0.144247,bagel,0.017988,191,189.0,17052.0,Key: 30\nFitness: None\nNodes:\n\t0 DefaultNod...,170.0,goldfish,0.114981


## Make Plot

In [6]:
# Downsample to every 500 generations
df_box = main_df[(main_df["saved_generation"]+1) % 500 == 0]

# Create a box and whiskers plot of scores for each generation
fig = px.box(
    df_box,
    x="saved_generation",
    y="score",
    title="Score Distribution over Generations (Every 500 Generations)",
    labels={"saved_generation": "Generation", "score": "Score"},
    width=800,
    height=800
)

fig.update_xaxes(
    tickmode="array",
    tickvals=[499, 999, 1499, 1999],
    ticktext=["500", "1000", "1500", "2000"]
)

fig.update_layout(template="plotly_white")
fig.show()


In [7]:
def bootstrap_median_ci(data, n_boot=100, alpha=0.05, random_state=run_config["seed"]):
    random_choice = np.random.RandomState(random_state)
    data = np.array(data)
    mean_of_medians = []
    # sample medians
    for _ in range(n_boot):
        sample = random_choice.choice(data, size=len(data), replace=True)
        mean_of_medians.append(np.mean(sample))
    mean_of_medians = np.sort(mean_of_medians)
    mean_val = np.mean(mean_of_medians)
    # set lower and upper bounds
    lower = np.percentile(mean_of_medians, 100 * alpha / 2)
    upper = np.percentile(mean_of_medians, 100 * (1 - alpha / 2))
    return mean_val, lower, upper

def compute_aggregated_df(run_ids, output_path, n_boot=1000, alpha=0.05, random_state= 11,
                          downsample_generations=[0, 499, 999, 1499, 1999]):
    """
    Loads and aggregates the score pickle files for the given run IDs.

    Parameters:
        run_ids (list): List of run ID strings.
        output_path (str): Base path where the run pickle files are stored.
        n_boot (int): Number of bootstrap iterations.
        alpha (float): Significance level for CI calculation.
        random_state (int): Random seed for bootstrapping.
        downsample_generations (list): Generations to keep for plotting.

    Returns:
        pd.DataFrame: Aggregated and downsampled DataFrame containing:
            - saved_generation
            - mean of medians across runs
            - ci_lower
            - ci_upper
            - plot_generation (saved_generation + 1)
    """
    df_list = []
    for rid in run_ids:
        pkl_path = os.path.join(output_path, rid, "classical", "score_agg_df.pkl")
        if os.path.exists(pkl_path):
            with open(pkl_path, "rb") as f:
                df_run = pickle.load(f)
            df_run["run_id"] = rid
            df_list.append(df_run)
        else:
            print(f"File not found: {pkl_path}")

    if not df_list:
        raise ValueError("No valid dataframes loaded for the given run IDs.")

    df_all = pd.concat(df_list, ignore_index=True)
    # Group score_q50 by generation (each group is a list of scores from different runs)
    grouped = df_all.groupby("saved_generation")["score_q50"].apply(list).reset_index()

    # Apply bootstrapping to compute median and confidence intervals.
    boot_results = grouped["score_q50"].apply(
        lambda x: bootstrap_median_ci(x, n_boot=n_boot, alpha=alpha, random_state=random_state)
    )
    grouped["median_across_runs"] = boot_results.apply(lambda x: x[0])
    grouped["ci_lower"] = boot_results.apply(lambda x: x[1])
    grouped["ci_upper"] = boot_results.apply(lambda x: x[2])
    grouped["plot_generation"] = grouped["saved_generation"] + 1

    # Downsample to the selected generations.
    df_plot = grouped[grouped["saved_generation"].isin(downsample_generations)].copy()
    df_plot["plot_generation"] = df_plot["saved_generation"] + 1
    return df_plot




In [8]:
def add_bootstrap_trace(fig, df_plot, label, line_color="royalblue", fill_color="rgba(65,105,225,0.2)"):
    """
    Adds a median line and a 95% bootstrapped confidence interval area to the Plotly figure.

    Parameters:
        fig (go.Figure): The Plotly figure.
        df_plot (pd.DataFrame): Aggregated data with columns:
            - plot_generation, median_across_runs, ci_lower, ci_upper.
        label (str): Label for the trace (used in the legend).
        line_color (str): Color for the median line.
        fill_color (str): Color for the CI shading.
    """
    median_trace_name = f"{label} Median Confidence"
    ci_trace_name = f"{label} 95% CI"

    # Median line trace.
    fig.add_trace(go.Scatter(
        x=df_plot["plot_generation"],
        y=df_plot["median_across_runs"],
        mode='lines+markers',
        name=median_trace_name,
        line=dict(color=line_color)
    ))

    # Confidence interval trace.
    fig.add_trace(go.Scatter(
        x=pd.concat([df_plot["plot_generation"], df_plot["plot_generation"][::-1]]),
        y=pd.concat([df_plot["ci_upper"], df_plot["ci_lower"][::-1]]),
        fill='toself',
        fillcolor=fill_color,
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        showlegend=True,
        name=ci_trace_name
    ))


### Adding Baseline

In [9]:
baseline_run_id = "20250330_163000_baseline"  # baseline_id
baseline_pkl_path = os.path.join(OUTPUT_PATH, baseline_run_id, "classical", "score_agg_df.pkl")
if os.path.exists(baseline_pkl_path):
    with open(baseline_pkl_path, "rb") as f:
        df_baseline = pickle.load(f)
    df_baseline["run_id"] = baseline_run_id
    df_baseline["plot_generation"] = df_baseline["saved_generation"] + 1
else:
    print(f"File not found: {baseline_pkl_path}")


In [ ]:
# Set run_ids
run_ids_pop100 = [
    "20250329_185329",
    "20250329_162400",
    "20250329_150935",
    "20250329_201700",
    "20250330_023000",
    "20250330_153700"
]

run_ids_pop50 = [
    "20250331_50_1",
    "20250331_50_2",
    "20250331_50_3",
    "20250331_50_4",
    "20250331_50_5",
    "20250331_50_6"
]

run_ids_pop10 = [
    "20250331_10_1",
    "20250331_10_2",
    "20250331_10_3",
    "20250331_10_4",
    "20250331_10_5",
    "20250331_10_6"
]

In [18]:
# Compute aggregated DataFrames for each batch
df_plot_pop100 = compute_aggregated_df(run_ids_pop100, OUTPUT_PATH, n_boot=100, alpha=0.05, random_state=11)
df_plot_pop50 = compute_aggregated_df(run_ids_pop50, OUTPUT_PATH, n_boot=100, alpha=0.05, random_state=11)
df_plot_pop10 = compute_aggregated_df(run_ids_pop10, OUTPUT_PATH, n_boot=100, alpha=0.05, random_state=11)


## Plotting

In [24]:
fig = go.Figure()

# Add traces for each batch with appropriate labels.
add_bootstrap_trace(fig, df_plot_pop100, label="Population 100",
                    line_color="royalblue", fill_color="rgba(65,105,225,0.2)")
add_bootstrap_trace(fig, df_plot_pop50, label="Population 50",
                    line_color="darkviolet", fill_color="rgba(204, 153, 255,0.2)")

add_bootstrap_trace(fig, df_plot_pop10, label="Population 10",
                    line_color="darkturquoise", fill_color="rgba(51, 204, 204,0.2)")

# Add the baseline trace
df_baseline_plot = df_baseline[df_baseline["saved_generation"].isin([0, 499, 999, 1499, 1999])].copy()
fig.add_trace(go.Scatter(
    x=df_baseline_plot["plot_generation"],
    y=df_baseline_plot["score_q50"],
    mode='lines+markers',
    name='Baseline Run Score',
    line=dict(color='darkgreen')
))

# Update the layout
fig.update_layout(
    title="AlexNet Median Performance Across Runs Compared to Baseline",
    xaxis_title="Generation",
    yaxis_title="Confidence",
    template="plotly_white"
)

# Set custom x-axis ticks
fig.update_xaxes(
    tickmode="array",
    tickvals=[0, 500, 1000, 1500, 2000],
    ticktext=["0", "500", "1000", "1500", "2000"]
)

# Display the figure
fig.show()
